In [1]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.2
    Uninstalling httpx-sse-0.4.2:
      Successfully uninstalled httpx-sse-0.4.2


In [2]:
import cohere
co = cohere.ClientV2("WuOIVkz19Iv8kNTzDDce4dYxUHIWuiQPqTw7hCI1") # Get your free API key: https://dashboard.cohere.com/api-keys

In [14]:
import os
import json
import numpy as np
import pandas as pd
from cohere import ClassifyExample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [5]:
def train_and_save():
  full_df = pd.read_json("./data/xed_with_embeddings.json", orient='index')
  df = full_df
  mlb = MultiLabelBinarizer()
  X = np.array(df.embeddings.tolist())
  y = mlb.fit_transform(df.labels_text)
  classes = mlb.classes_
  classes_mapping = {index: emotion for index, emotion in enumerate(mlb.classes_)}
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
  base_lr = LogisticRegression(solver='lbfgs', random_state=0)
  chain = ClassifierChain(base_lr, order='random', random_state=0)
  chain.fit(X_train, y_train)
  print(chain.score(X_test, y_test))
  pickle.dump(chain, open("./data/models/emotion_chain.pkl", 'wb'))

In [8]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.4 MB/s eta 0:00:00


In [23]:
# ===========================================
# 🪄 1. Cài đặt & nâng cấp Cohere SDK
# ===========================================
!pip install --upgrade cohere

import cohere

# ===========================================
# 🔐 2. Khởi tạo client với API key
# ===========================================
API_KEY = "WuOIVkz19Iv8kNTzDDce4dYxUHIWuiQPqTw7hCI1"   # ⚠️ Thay bằng API key của bạn
co = cohere.Client(API_KEY)

# ===========================================
# 🧠 3. Hàm phân tích cảm xúc
#    - Không dùng `messages` nữa, dùng `input`
# ===========================================
def sentiment_analysis(text):
    prompt = f"""Phân tích cảm xúc của đoạn văn sau và trả về một trong 3 nhãn:
- positive (tích cực)
- negative (tiêu cực)
- neutral (trung lập)

Đoạn văn: "{text}"
Trả lời chỉ bằng 1 từ: positive, negative hoặc neutral.
"""
    response = co.chat(
        model="command-a-03-2025",
        input=prompt   # ✅ cú pháp đúng cho SDK hiện tại
    )
    return response.text.strip().lower()

# ===========================================
# 📝 4. Danh sách văn bản cần phân tích
# ===========================================
texts = [
    "Tôi rất thích sản phẩm này!",
    "The service was terrible, I'm disappointed.",
    "Không biết nói gì, bình thường thôi.",
    "This product is amazing and works perfectly.",
    "Dịch vụ quá tệ, tôi sẽ không quay lại.",
    "Mọi thứ ổn, không có gì đặc biệt."
]

# ===========================================
# 📊 5. Phân tích & in kết quả
# ===========================================
for t in texts:
    sentiment = sentiment_analysis(t)
    print(f"Text: {t}")
    print(f"Sentiment: {sentiment}")
    print("-" * 40)


TypeError: BaseCohere.chat() got an unexpected keyword argument 'input'

In [24]:
!pip install transformers torch --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

In [25]:
MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [26]:
def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = outputs.logits[0].detach().numpy()
        probs = np.exp(scores) / np.sum(np.exp(scores))
        label_id = np.argmax(probs)

    labels = ['negative', 'neutral', 'positive']
    return labels[label_id], probs


In [27]:
texts = [
    "Tôi rất thích sản phẩm này!",
    "The service was terrible, I'm disappointed.",
    "Không biết nói gì, bình thường thôi.",
    "This product is amazing and works perfectly.",
    "Dịch vụ quá tệ, tôi sẽ không quay lại.",
    "Mọi thứ ổn, không có gì đặc biệt."
]

for t in texts:
    label, probs = sentiment_analysis(t)
    print(f"Text: {t}")
    print(f"Sentiment: {label} | Probabilities: {probs}")
    print("-" * 40)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Tôi rất thích sản phẩm này!
Sentiment: positive | Probabilities: [0.01720183 0.08912598 0.8936722 ]
----------------------------------------
Text: The service was terrible, I'm disappointed.
Sentiment: negative | Probabilities: [0.9597782  0.02790626 0.01231556]
----------------------------------------
Text: Không biết nói gì, bình thường thôi.
Sentiment: neutral | Probabilities: [0.3483161  0.45180523 0.19987863]
----------------------------------------
Text: This product is amazing and works perfectly.
Sentiment: positive | Probabilities: [0.01309985 0.05147919 0.93542093]
----------------------------------------
Text: Dịch vụ quá tệ, tôi sẽ không quay lại.
Sentiment: negative | Probabilities: [0.8725266  0.09646463 0.03100877]
----------------------------------------
Text: Mọi thứ ổn, không có gì đặc biệt.
Sentiment: neutral | Probabilities: [0.14867358 0.45658904 0.3947374 ]
----------------------------------------
